In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import nltk

In [3]:
train_data = pd.read_csv('C:\\AADeloitte\\learning\\Boom\\Python Practise files\\Python\\Sentiment Analysis\\train.csv')
test_data = pd.read_csv('C:\\AADeloitte\\learning\\Boom\\Python Practise files\\Python\\Sentiment Analysis\\test.csv')

In [4]:
print(train_data.shape, test_data.shape)
train_data.head()

(31962, 3) (17197, 2)


,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
train_data['source'], test_data['source'] = 'train', 'test'

In [6]:
train_data.label.value_counts(dropna=False)

0    29720
1     2242
Name: label, dtype: int64

In [8]:
train_data.label.value_counts(dropna=False).div(len(train_data)) # proportion of non-racist tweets

0    0.929854
1    0.070146
Name: label, dtype: float64

In [9]:
print(train_data.isnull().sum(),'\n',test_data.isnull().sum()) # checking for missing values

id        0
label     0
tweet     0
source    0
dtype: int64 
 id        0
tweet     0
source    0
dtype: int64


In [10]:
test_data.head() # checking the first five values of the dataset

,id,tweet,source
0,31963,#studiolife #aislife #requires #passion #dedic...,test
1,31964,@user #white #supremacists want everyone to s...,test
2,31965,safe ways to heal your #acne!! #altwaystohe...,test
3,31966,is the hp and the cursed child book up for res...,test
4,31967,"3rd #bihday to my amazing, hilarious #nephew...",test


In [11]:
[i for i in train_data.loc[:2, 'tweet']]

[' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run',
 "@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked",
 '  bihday your majesty']

In [ ]:
# from nltk.tokenize import TweetTokenizer

In [16]:
# [re.findall(pattern= r'(https?://\w.*\s?)', string=x) for x in train_data.loc[:,'tweet']]


In [12]:
hastags = [re.findall(pattern= r'(#\w+\s?)', string=x) for x in train_data.loc[:,'tweet']]

In [13]:
hastags_list = []
for i in hastags:
    for j in i:
        hastags_list.append(j.strip())

In [14]:
hastags_count = Counter(hastags_list)

In [15]:
hastags_count.most_common(10)

[('#love', 1542),
 ('#positive', 874),
 ('#healthy', 571),
 ('#smile', 548),
 ('#thankful', 491),
 ('#fun', 434),
 ('#life', 406),
 ('#summer', 367),
 ('#model', 365),
 ('#affirmation', 363)]

In [17]:
tknzr = nltk.TweetTokenizer()

In [18]:
pattern = r'[@#]\w+'

In [19]:
re.sub(pattern, ' ', train_data.tweet[0])

'   when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.    '

In [25]:
cleaned_sent = [re.sub(r'[^A-Za-z]', ' ', x).lower() for x in train_data['tweet'][:10]]

In [29]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [30]:
ps = PorterStemmer()

In [31]:
cleaned_sent = [ps.stem(word) for word in cleaned_sent if word not in set(stopwords.words('english'))]

In [32]:
cleaned_sent = " ".join(cleaned_sent)

In [34]:
cleaned_sent

'  user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction     run  user  user thanks for  lyft credit i can t use cause they don t offer wheelchair vans in pdx      disapointed  getthank   bihday your majesti  model   i love u take with u all the time in ur                                        factsguide  society now     motiv       huge fan fare and big talking before they leave  chaos and pay disputes when they get there   allshowandnogo     user camping tomorrow  user  user  user  user  user  user  user danny    the next school year is the year for exams      can t think about that       school  exams    hate  imagine  actorslife  revolutionschool  girl we won    love the land     allin  cavs  champions  cleveland  clevelandcavaliers         user  user welcome here    i m   it s so  gr    '

In [35]:
list_corpus = []

In [36]:
for i in range(0, len(train_data)):
    review = re.sub(pattern= r'[@]\w+',repl= ' ',string=train_data['tweet'][i])
    review = re.sub(pattern= r'[^a-zA-Z]',repl= ' ', string=review)
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    list_corpus.append(review)  

In [37]:
list_corpus[:5]

['father dysfunct selfish drag kid dysfunct run',
 'thank lyft credit use caus offer wheelchair van pdx disapoint getthank',
 'bihday majesti',
 'model love u take u time ur',
 'factsguid societi motiv']

In [38]:
from sklearn.feature_extraction.text import CountVectorizer

In [39]:
cv = CountVectorizer(ngram_range=(2,2), max_features=6500)

In [40]:
X = cv.fit_transform(list_corpus).toarray()

In [41]:
from sklearn.model_selection import train_test_split

In [42]:
y = train_data.label.values

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 12, test_size=0.2, stratify=y)

In [44]:
X_train.shape

(25569, 6500)

In [45]:
from sklearn.naive_bayes import GaussianNB

In [46]:
nv_class = GaussianNB()

In [47]:
nv_class.fit(X, y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [48]:
nv_pred = nv_class.predict(X_test)

In [49]:
from sklearn import metrics

In [50]:
metrics.confusion_matrix(y_test, nv_pred)

array([[3551, 2394],
       [   0,  448]], dtype=int64)

In [52]:
metrics.accuracy_score(y_test, nv_pred)

0.6255279211637729